In [2]:
!pip install torch torchvision

In [10]:
import torch
import torchvision
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd

In [4]:
#Configuration
test_dir= '/kaggle/input/dogs-vs-cats/test1/test1'
train_dir= '/kaggle/input/dogs-vs-cats/train/train'
submission_file = "/kaggle/working/submission.csv"

#### Loading Dataset Class

In [39]:
class CatsDogsDataset(Dataset):
    def __init__(self, data_dir, transform= None, mode='train'):
        self.data_dir= data_dir
        self.transform= transform
        self.mode= mode
        self.image_files= os.listdir(data_dir)

    def __len__(self):
        return(len(self.image_files))

    def __getitem__(self, idx):
        img_name= self.image_files[idx]
        img_path = os.path.join(self.data_dir, img_name)
        image= Image.open(img_path).convert("RGB")

        if self.mode== 'train':
            label=0 if 'cat' in img_name else 1
        else:
            label= int(img_name.split(".")[0])

        if self.transform:
            image= self.transform(image)

        return image,label
        

#### Transforms

In [40]:
transform= transforms.Compose([transforms.Resize((64, 64)),transforms.ToTensor()])

#### Load Datasets

In [41]:
train_dataset= CatsDogsDataset(train_dir, transform= transform, mode='train')
test_dataset= CatsDogsDataset(test_dir, transform= transform, mode= 'test')

train_loader= DataLoader(train_dataset, batch_size=10, shuffle='True')
test_loader= DataLoader(test_dataset, batch_size=10, shuffle= 'False')

#### Simple FNN

In [42]:
class simpleFNN(nn.Module):

    def __init__(self):
        super(simpleFNN, self).__init__()
        self.fc1 = nn.Linear(64 * 64* 3, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x= x.view(x.size(0), -1)
        x= torch.relu(self.fc1(x))
        x= torch.relu(self.fc2(x))
        x= torch.sigmoid(self.fc3(x))

        return x

#### Initialize the model, loss function, and optimizer

In [43]:
model= simpleFNN()
criterion= nn.BCELoss()
optimizer= optim.Adam(model.parameters(), lr=0.001)

#### Training

In [44]:
num_epochs = 5  # Number of training epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        labels = labels.float().view(-1, 1)  # Reshape labels for BCELoss

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.6934
Epoch [2/5], Loss: 0.6863
Epoch [3/5], Loss: 0.6780
Epoch [4/5], Loss: 0.6670
Epoch [5/5], Loss: 0.6603


#### Predict on the test set

In [45]:
model.eval()  # Set model to evaluation mode
predictions = []

with torch.no_grad():  # Disable gradient calculation
    for images, ids in test_loader:
        outputs = model(images)
        predicted = (outputs > 0.5).float()  # Convert probabilities to binary predictions
        predictions.extend(list(zip(ids.numpy(), predicted.view(-1).numpy())))

In [46]:
print(predictions[:100])

[(4897, 0.0), (2507, 1.0), (3020, 0.0), (9263, 1.0), (8833, 0.0), (7892, 1.0), (1070, 0.0), (1653, 1.0), (5289, 1.0), (7244, 1.0), (77, 0.0), (1932, 1.0), (5656, 0.0), (8857, 0.0), (7326, 1.0), (1411, 0.0), (2936, 1.0), (4716, 1.0), (5706, 0.0), (5793, 0.0), (9648, 1.0), (10629, 1.0), (11784, 0.0), (3917, 1.0), (2315, 0.0), (2129, 1.0), (6186, 1.0), (7901, 1.0), (10168, 0.0), (7549, 1.0), (3726, 1.0), (8773, 0.0), (2875, 0.0), (4788, 1.0), (8930, 0.0), (1069, 1.0), (9635, 0.0), (5013, 1.0), (10856, 1.0), (1309, 0.0), (6924, 0.0), (4833, 1.0), (12004, 0.0), (6548, 0.0), (3523, 0.0), (538, 1.0), (10419, 1.0), (10911, 0.0), (8012, 0.0), (10526, 1.0), (8330, 0.0), (7567, 1.0), (1351, 1.0), (7204, 1.0), (382, 0.0), (4130, 1.0), (9179, 1.0), (4704, 0.0), (8187, 1.0), (2476, 1.0), (6862, 0.0), (8963, 1.0), (10566, 1.0), (3232, 1.0), (7223, 1.0), (6146, 0.0), (6055, 1.0), (3117, 1.0), (4935, 0.0), (11000, 1.0), (3889, 1.0), (8070, 1.0), (6677, 0.0), (11869, 1.0), (7216, 0.0), (5515, 1.0), (108

#### Save predictions to a CSV file

In [47]:
submission = pd.DataFrame(predictions, columns=["id", "label"])
submission.to_csv(submission_file, index=False)
print("Predictions saved to submission.csv")

Predictions saved to submission.csv
